## Introduction to Object Detection with YOLOv8

## Demo Outline
1. Setting up dependencies and libraries
2. Loading and demo on pre-trained model
3. Dataset Contents
4. Training a model from scratch
5. Evaluation and metrics

## Dependencies and Libraries
The following is the list of libraries you need to install:
- Python (version >= 3.8)
- [PyTorch (CUDA version / CPU-Only)](https://pytorch.org/get-started/locally/)
    - Torchvision
    - Torchaudio
- [CUDA](https://developer.nvidia.com/cuda-toolkit-archive)
    - [Checking GPU compatibility](https://developer.nvidia.com/cuda-gpus)
- [Ultralytics](https://docs.ultralytics.com/quickstart/)
    - Alternative (For Development Version): [Github Repo](https://github.com/ultralytics/ultralytics)

### Importing Libraries

In [ ]:
from ultralytics import YOLO
import torch
print("CUDA is available:", torch.cuda.is_available())

# For Visualizations
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

## Pre-Trained Model
- [List of available models](https://docs.ultralytics.com/models/)

In [ ]:
# Loading a model
model = YOLO(model="yolov8n.pt")

#### Sample Inference

In [ ]:
# Visualize Sample Original Image
img = cv2.imread("sample1.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.axis('off')
plt.title("Original")
plt.show()

In [ ]:
# Perform prediction on two images
results = model.predict(["sample1.jpg","sample2.jpg"],
                       save=True,
                       exist_ok=True,
                       project="predict",
                       name="demo",
                       verbose=False)

# View contents of results
print(results)

In [ ]:
# Get detections of the first image
boxes = results[0].boxes
img_det = img.copy()

for box in boxes:
    x1, y1, x2, y2 = box.xyxy[0]
    label = results[0].names[int(box.cls[0])]

    # Drawing of Bounding Boxes and Labels
    cv2.rectangle(img_det, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
    cv2.putText(img_det, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

In [ ]:
plt.imshow(img_det)
plt.axis('off')
plt.title("Image w/ Detections")
plt.show()

In [ ]:
# Visualization automatically generated by Ultralytics
img_det = cv2.imread(os.path.join('predict', 
                                  'demo',
                                  'sample1.jpg'))
img_det = cv2.cvtColor(img_det, cv2.COLOR_BGR2RGB)

plt.imshow(img_det)
plt.axis('off')
plt.title("Image w/ Detections")
plt.show()

## Loading a Dataset
- For the datasets used in this workshop, you can download them here: [datasets.zip](https://drive.google.com/file/d/1z5usegJoqEK7GZET4W9dudU2nszbjluA/view?usp=sharing)

### Ultralytics Datasets
- [List of available datasets](https://docs.ultralytics.com/datasets/)

In [ ]:
# Sample Use (This will download the dataset)
# model.train(data="coco8.yaml")

### Custom Dataset

In [ ]:
# Print Folder Structure of Custom Dataset
def list_files(startpath):
    for root, _, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        dash = '-' * (level + 1)
        print('{}{}'.format(dash, os.path.basename(root)))
        for f in files[:1]:
            print('{}{}'.format(dash + '-', f))

list_files("datasets/african-wildlife")

In [ ]:
# Sample Label File
with open("datasets/african-wildlife/train/labels/1 (43).txt", "r") as file:
    lines = file.readlines()

print("Sample Annotations: \n")
for line in lines:
    print(line)

In [ ]:
# Sample .yaml file
with open("datasets/african-wildlife/data.yaml", "r") as file:
    lines = [line.strip() for line in file.readlines()]
    
for line in lines:
    print(line)

## YOLOv8 Model From Scratch

### Training

In [ ]:
# Loading of Model without Pre-Trained Weights
model = YOLO(model="yolov8n.yaml")

In [ ]:
# Train Function
metrics = model.train(data=os.path.join("datasets", # Dataset to be Trained
                                        "african-wildlife",
                                        "data.yaml"), 
                                        
                      val=False, # Performing Validation Every Epoch

                      cache=True, # Cache on True/`ram`, `disk` or False
                      device=0, # Available for multi-GPUs (eg. 0,1) or Apple chips (mps)
                        
                      #--- HYPERPARAMETERS ---#
                      epochs=5, # Training epochs
                      batch=32, # Batch size, auto mode if set to decimal 

                      optimizer="Adam", # has auto option
                      lr0=0.001, # initial learning rate
                      lrf=0.01, # final learning rate = lr0 * lrf
                      momentum=0.9, # momentum factor
                      

                      #--- SAMPLE AUGMENTATIONS ---#
                      imgsz=640, # Resizes Images
                      translate=0.1, # Moves the image a fraction of the image size
                      scale=0.5, # Scales image by a factor
                      mosaic=1.0, # Combines 4 training images into one
                      mixup=1.0, # Blends two images and their labels
                      
                        
                      #--- SAVING ---#
                      save=True, 
                      save_period=1, # checkpoint saves

                      seed=123, # Reproduce Results
                      exist_ok=True,
                      project="train", # Training Directory
                      name="yolov8n_wildlife", # Save Folder Name
                      )

In [ ]:
# For Custom Training/Testing Loop
model.model.train()
model.model.eval()

In [ ]:
# Resuming Training from Last Save
model = YOLO(os.path.join("train",
                          "yolov8n_wildlife",
                          "weights",
                          "last.pt"))
results = model.train(resume=True)

### Evaluation

In [ ]:
# Loading best model from training
model = YOLO(os.path.join("train",
                          "yolov8n_wildlife",
                          "weights",
                          "best.pt"))

In [ ]:
# Perform Evaluation
metrics = model.val(data=os.path.join("datasets",
                                      "african-wildlife",
                                      "data.yaml"),
                    split="test",
                    batch=32,
                    conf=0.001, # Score Threshold
                    iou=0.6, # IoU threshold for NMS
                    max_det=300, # Maximum number of detections
                    exist_ok=True,
                    project="eval",
                    name="yolov8n_wildlife")

In [ ]:
# Getting Scores From Metrics Object
print('mAP50-95:', metrics.box.map)
print('mAP50:', metrics.box.map50)
print('mAP75:', metrics.box.map75)
print('mAP50-95 per class:', metrics.box.maps)

In [ ]:
# Confusion Matrix Generated
img = cv2.imread(os.path.join("eval",
                              "yolov8n_wildlife",
                              "confusion_matrix.png"))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.title("Original")
plt.show()

### Extracting Predictions

In [ ]:
# Loading best model from training
model = YOLO(os.path.join("train",
                          "yolov8n_wildlife",
                          "weights",
                          "best.pt"))

In [ ]:
# Getting Detections
results = model.predict(source=os.path.join("datasets",
                                      "african-wildlife",
                                      "test",
                                      "images"),
                        conf=0.1, # Score Threshold
                        iou=0.5, # IoU threshold for NMS
                        max_det=300, # Maximum number of detections
                        save_txt=True, # Bounding box predictions and labels
                        save=True, # Visualization
                        verbose=False,
                        exist_ok=True,
                        project="predict",
                        name="yolov8n_wildlife")


In [ ]:
import random as rnd

# Get detections of a random image
index = 4 # rnd.randint(0, len(results))
img = cv2.imread(results[index].path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
boxes = results[index].boxes

for box in boxes:
    x1, y1, x2, y2 = box.xyxy[0]
    label = results[0].names[int(box.cls[0])]
    
    # Drawing of Bounding Boxes and Labels
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
    cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.title("Image w/ Detections")
plt.show()

In [ ]:
# Visualization automatically generated by Ultralytics
predict_dir = os.listdir(os.path.join('predict', 'yolov8n_wildlife'))

img_det = cv2.imread(os.path.join('predict', 'yolov8n_wildlife', predict_dir[index]))
img_det = cv2.cvtColor(img_det, cv2.COLOR_BGR2RGB)

plt.imshow(img_det)
plt.axis('off')
plt.title("Image w/ Detections")
plt.show()

## Useful References
- [PyTorch and Virtual Environment Setup](https://www.youtube.com/watch?v=GMSjDTU8Zlc)
- [List of YOLO Train Settings](https://docs.ultralytics.com/modes/train/#train-settings)
- [List of YOLO Val Settings](https://docs.ultralytics.com/modes/val/#arguments-for-yolo-model-validation)
- [List of YOLO Predict Settings](https://docs.ultralytics.com/modes/predict/#inference-arguments)